In [60]:
!git clone https://github.com/PonyGE/PonyGE2.git

fatal: destination path 'PonyGE2' already exists and is not an empty directory.


In [61]:
!cd PonyGE2/ && git pull

Already up to date.


In [62]:
#pip installs

%pip install numpy

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: numpy in c:\users\hsabi\appdata\local\programs\python\python311\lib\site-packages (1.24.1)




[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [63]:
%pip install sewar



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
%pip install openpyxl


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
%%capture captured
import shutil
import os

experiment = "regression"

shutil.copy(f"./fitness/{experiment}.py", "./PonyGE2/src/fitness/")
!cd ./PonyGE2/src && python ponyge.py --parameters ../../parameters/{experiment}.txt
os.remove(f"./PonyGE2/src/fitness/{experiment}.py")


In [31]:
import numpy as np
from sewar.full_ref import mse
from openpyxl import load_workbook

captured_str = str(captured).split('\n')
for line in captured_str:
    if "Phenotype: " in line:
        phenotype = line.replace("Phenotype: ", "").strip()
    elif "DATASET_NAME" in line:
        dataset_name = line.replace("DATASET_NAME: ", "")

dataset_path = './datasets/data.xlsx'



In [32]:
import torch
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def extremeVal(reference, val):
    if (val > 1e2*reference) or (val < -1e2*reference):
        return 0
    else:
        return val

split_type_index = {"arima": 2, "sarima": 1}
models_values_column = {
    "arima": {
        "series": 1,
        "arima": 3,
        "mlp": 4,
        "svr": 5,
        "rbf": 6,
    },
    "sarima": {
        "series": 6,
        "sarima": 2,
        "mlp": 3,
        "svr": 4,
        "rbf": 5,
    },
}
window_size = 5

def load_data(dataset_name, dataset_path, linear_type):
    wb = load_workbook(dataset_path)
    dataset_name_arr = [dataset_name]

    dataset = {}
    for name in dataset_name_arr:

        ws = wb[name]
        columns = [ws["A"], ws["B"], ws["C"],
                    ws["D"], ws["E"], ws["F"], ws["G"]]

        sheet = [[] for _ in range(len(columns[0]))]

        for c in columns:
            for index, item in enumerate(c):
                sheet[index].append(item.value)

        sheet = np.array(sheet)
        dataset[name] = {"raw": sheet}

    for name, _ in dataset.items():
        sheet = dataset[name]["raw"]

        dataset[name]["series"] = {
            "treino": [], "teste": [], "validacao": []}
        dataset[name][linear_type] = {
            "treino": [], "teste": [], "validacao": []}
        dataset[name]["mlp"] = {"treino": [], "teste": [], "validacao": []}
        dataset[name]["svr"] = {"treino": [], "teste": [], "validacao": []}
        dataset[name]["rbf"] = {"treino": [], "teste": [], "validacao": []}

        for row in sheet[1:]:
            split_type = row[split_type_index[linear_type]]
            if linear_type == "arima" and row[4] == None:
                continue
            key = None
            if split_type.lower() == "teste":
                key = "teste"
            elif split_type.lower() == "validacao" or split_type.lower() == "validação":
                key = "validacao"
            elif split_type.lower() == "treinamento":
                key = "treino"
            else:
                continue

            for model_key, val in models_values_column[linear_type].items():
                dataset[name][model_key][key].append(float(row[val]))

        for type_data in ["teste", "treino", "validacao"]:
            dataset[name]["series"][type_data] = np.array(
                dataset[name]["series"][type_data])
            dataset[name][linear_type][type_data] = np.array(
                dataset[name][linear_type][type_data])
            dataset[name]["mlp"][type_data] = np.array(
                dataset[name]["mlp"][type_data])
            dataset[name]["svr"][type_data] = np.array(
                dataset[name]["svr"][type_data])
            dataset[name]["rbf"][type_data] = np.array(
                dataset[name]["rbf"][type_data])

    fixExtreme = np.vectorize(extremeVal)

    for name in dataset.keys():
        reference = np.max(dataset[name]["series"]["treino"])

        for type_data in ["teste", "treino", "validacao"]:
            dataset[name]["mlp"][type_data] = fixExtreme(
                reference, dataset[name]["mlp"][type_data])
            dataset[name]["svr"][type_data] = fixExtreme(
                reference, dataset[name]["svr"][type_data])
            dataset[name]["rbf"][type_data] = fixExtreme(
                reference, dataset[name]["rbf"][type_data])
            dataset[name][linear_type][type_data] = fixExtreme(
                reference, dataset[name][linear_type][type_data])

    return dataset[dataset_name]

def build_model(phenotype):
    model = {"linear": {}, "nlinear": {}}

    linear, nlinear = phenotype.split(';')

    weight_tuple, linear_model = linear.split(':')

    weight_values = weight_tuple.removeprefix('(').removesuffix(')')
    model["linear"][linear_model] = [
        float(x) for x in weight_values.split(' ')]

    nlinear_parts = nlinear.split(',')

    for i in range(len(nlinear_parts)):
        weight_tuple, nlinear_model = nlinear_parts[i].split(':')
        weight_values = weight_tuple.removeprefix('(').removesuffix(')')

        if nlinear_model in model["nlinear"]:
            model["nlinear"][nlinear_model] += [
                float(x) for x in weight_values.split(' ')]
        else:
            model["nlinear"][nlinear_model] = [
                float(x) for x in weight_values.split(' ')]

    return model

def predict_mse(model, arima_dataset, sarima_dataset, split):
    if "arima" in model["linear"].keys():
        dataset = arima_dataset
        linear_type = "arima"
    else:
        dataset = sarima_dataset
        linear_type = "sarima"
    predict = dataset["series"][split]
    X_train = apply_window(
        window_size, split, linear_type, dataset, model_names=model["nlinear"].keys())
    kernel = []
    for i in range(window_size):
        for _, val in model["linear"].items():
            kernel.append(val[i])
        for _, val in model["nlinear"].items():
            kernel.append(val[i])
    return mse(create_prediction(X_train, np.reshape(kernel, (1, -1))), predict)

def apply_window(window_size, data_type, linear_type, dataset, model_names=None):
    size_pred = len(dataset["series"][data_type])
    new_data = []
    for w in range(window_size):
        linear = np.concatenate(
            [np.zeros(w), dataset[linear_type][data_type]])
        linear = linear[:size_pred]
        row = np.array(linear)
        for model_name in model_names:
            nonlinear = np.concatenate(
                [np.zeros(w), dataset[model_name][data_type]])
            nonlinear = nonlinear[:size_pred]
            row = np.column_stack([row, nonlinear])
        if len(new_data) == 0:
            new_data = row
        else:
            new_data = np.column_stack([new_data, row])
    return np.array(new_data)

def create_prediction(X, kernel):
    X = torch.tensor(np.expand_dims(X, axis=(0, 1))).float()
    kernel = torch.tensor(np.expand_dims(kernel, axis=(0, 1))).float()
    X.to(device)
    kernel.to(device)
    result = F.conv2d(X, kernel)
    return result.numpy().squeeze().astype("f")


In [36]:
print(f'Dataset: {dataset_name}')
print(f'Fenótipo: {phenotype}')

arima_dataset = load_data(dataset_name, './datasets/arima_data.xlsx', 'arima')
sarima_dataset = load_data(dataset_name, './datasets/sarima_data.xlsx', 'sarima')
model = build_model(phenotype)
print('Fitness treino:', f"{predict_mse(model, arima_dataset, sarima_dataset, 'treino'):.3e}")
print('Fitness teste:', f"{predict_mse(model, arima_dataset, sarima_dataset, 'teste'):.3e}")
print('Fitness validacao:', f"{predict_mse(model, arima_dataset, sarima_dataset, 'validacao'):.3e}")


Dataset: B1H
Fenótipo:   (0.0 0 1 0 0 05326067.5 88 44859.82 0922.77 647):sarima;(2 0 0 9 0.035 2 1 5859.1 15 64):rbf


ValueError: could not convert string to float: ''